<a href="https://www.kaggle.com/code/vanha2301/voiceover-dubbing?scriptVersionId=284240018" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Cài đặt thư viện và Tải video mẫu

# 1. Cài đặt thư viện faster-whisper (phiên bản chạy nhanh trên GPU)
!pip install faster-whisper

from faster_whisper import WhisperModel

model_size = "large-v3"         # Mô hình xịn nhất
model = WhisperModel(model_size, device="cuda", compute_type="float16")
print("✅ Đã cài đặt xong và tải video mẫu thành công!")

In [1]:
!pip install faster-whisper

from faster_whisper import WhisperModel

model_size = "large-v3"         # Mô hình xịn nhất
model = WhisperModel(model_size, device="cuda", compute_type="float16")
print("✅ Đã cài đặt xong và tải video mẫu thành công!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 MB 51.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.8/37.8 MB 54.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 101.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.4 MB/s eta 0:00:00


vocabulary.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

✅ Đã cài đặt xong và tải video mẫu thành công!


## Code xử lý (Core Logic)

In [2]:
import os
import time
from faster_whisper import WhisperModel

# --------- CẤU HÌNH ---------
dataset_dir = "/kaggle/input/toiyeusuhopnhat"   # thư mục chứa tap1.mp4, tap2.mp4, ...
output_dir = "/kaggle/working"                      # nơi lưu các file .txt
model_size = "large-v3"                            # hoặc "medium" nếu muốn nhẹ hơn

# Tạo thư mục output nếu chưa có
os.makedirs(output_dir, exist_ok=True)

print(f"🚀 Đang load mô hình {model_size} vào GPU T4...")
model = WhisperModel(
    model_size,
    device="cuda",          # hoặc "cpu" nếu không có GPU
    compute_type="float16"  # với GPU T4 dùng float16 là hợp lý
)

# Lấy danh sách tất cả file video trong dataset_dir
# Có thể thêm/lượt bỏ đuôi tùy nhu cầu
video_extensions = (".mp4", ".mkv", ".mov", ".avi", ".wav", ".mp3")

video_files = [
    f for f in os.listdir(dataset_dir)
    if f.lower().endswith(video_extensions)
]

# Sắp xếp cho đẹp (tap1, tap2, tap3, ...)
video_files.sort()

if not video_files:
    print("❌ Không tìm thấy file video nào trong:", dataset_dir)
else:
    print("📂 Tìm thấy các file:")
    for vf in video_files:
        print(" -", vf)

    # --------- VÒNG LẶP XỬ LÝ TỪNG FILE ---------
    for filename in video_files:
        video_path = os.path.join(dataset_dir, filename)
        base_name, _ = os.path.splitext(filename)          # tap1.mp4 -> ("tap1", ".mp4")
        output_file = os.path.join(output_dir, base_name + ".txt")

        print("\n" + "=" * 80)
        print(f"⏳ Đang bắt đầu xử lý: {filename}")
        start_time = time.time()

        # Transcribe
        segments, info = model.transcribe(
            video_path,
            beam_size=5,
            vad_filter=True,
            vad_parameters=dict(min_silence_duration_ms=500)
        )

        print(f"\nDetected language: {info.language} "
              f"(Độ tin cậy: {info.language_probability:.2f})")
        print("-" * 50)
        print(f"{'TIME START':<10} | {'TIME END':<10} | {'TEXT CONTENT'}")
        print("-" * 50)

        full_text_log = ""

        # Ghi từng segment ra file .txt
        with open(output_file, "w", encoding="utf-8") as f:
            for segment in segments:
                start = f"{segment.start:.2f}s"
                end = f"{segment.end:.2f}s"
                text = segment.text.strip()

                # In ra màn hình
                print(f"{start:<10} | {end:<10} | {text}")

                # Ghi ra file (định dạng tùy bạn)
                f.write(start + "\n")
                f.write(end + "\n")
                f.write(text + "\n\n")

                full_text_log += text + " "

        end_time = time.time()
        print("-" * 50)
        print(f"✅ Xử lý xong {filename} trong: {end_time - start_time:.2f} giây")
        print(f"📝 Đã lưu file: {output_file}")
        # Nếu không cần in full_text_log thì có thể bỏ
        # print("\nTOÀN BỘ VĂN BẢN GỘP:")
        # print(full_text_log.strip())

    print("\n🎉 Đã xử lý xong toàn bộ dataset!")


🚀 Đang load mô hình large-v3 vào GPU T4...
📂 Tìm thấy các file:
 - ToiYeuSuHopNhatTap1.mp4
 - ToiYeuSuHopNhatTap10.mp4
 - ToiYeuSuHopNhatTap11.mp4
 - ToiYeuSuHopNhatTap12.mp4
 - ToiYeuSuHopNhatTap13.mp4
 - ToiYeuSuHopNhatTap14.mp4
 - ToiYeuSuHopNhatTap15.mp4
 - ToiYeuSuHopNhatTap16.mp4
 - ToiYeuSuHopNhatTap17.mp4
 - ToiYeuSuHopNhatTap18.mp4
 - ToiYeuSuHopNhatTap19.mp4
 - ToiYeuSuHopNhatTap2.mp4
 - ToiYeuSuHopNhatTap20.mp4
 - ToiYeuSuHopNhatTap21.mp4
 - ToiYeuSuHopNhatTap23.mp4
 - ToiYeuSuHopNhatTap24.mp4
 - ToiYeuSuHopNhatTap25.mp4
 - ToiYeuSuHopNhatTap26.mp4
 - ToiYeuSuHopNhatTap27.mp4
 - ToiYeuSuHopNhatTap28.mp4
 - ToiYeuSuHopNhatTap29.mp4
 - ToiYeuSuHopNhatTap30.mp4
 - ToiYeuSuHopNhatTap31.mp4
 - ToiYeuSuHopNhatTap32.mp4
 - ToiYeuSuHopNhatTap33.mp4
 - ToiYeuSuHopNhatTap34.mp4
 - ToiYeuSuHopNhatTap35.mp4
 - ToiYeuSuHopNhatTap36.mp4
 - ToiYeuSuHopNhatTap37.mp4
 - ToiYeuSuHopNhatTap38.mp4
 - ToiYeuSuHopNhatTap39.mp4
 - ToiYeuSuHopNhatTap3phan1.mp4
 - ToiYeuSuHopNhatTap3phan2.mp4
 - Toi

In [2]:
import time
from faster_whisper import WhisperModel

# --- CẤU HÌNH ---
video_path = "" # Đường dẫn video (thay bằng file của bạn sau này)
output_file = "/kaggle/working/tua_lai_tuyet_vong_tap3.txt"
model_size = "large-v3"         # Mô hình xịn nhất
# model_size = "medium"         # Dùng cái này nếu muốn test nhanh hơn nữa

print(f"🚀 Đang load mô hình {model_size} vào GPU T4...")
u
print("⏳ Đang bắt đầu xử lý video (Transcribing)...")
start_time = time.time()

# --- XỬ LÝ ---
# beam_size=5: Giúp AI dò tìm câu chính xác hơn
segments, info = model.transcribe(
    video_path, 
    beam_size=5,
    vad_filter=True,
    vad_parameters=dict(min_silence_duration_ms=500)
)

# --- HIỂN THỊ KẾT QUẢ ---
print(f"\nDetected language: {info.language} (Độ tin cậy: {info.language_probability:.2f})")
print("-" * 50)
print(f"{'TIME START':<10} | {'TIME END':<10} | {'TEXT CONTENT'}")
print("-" * 50)

full_text_log = ""
print(video_path)
# segments là một generator (nó chạy đến đâu nhả chữ đến đó)
with open(output_file, "w", encoding="utf-8") as f:
    for segment in segments:
        # Định dạng thời gian cho đẹp
        start = f"{segment.start:.2f}s"
        end = f"{segment.end:.2f}s"
        text = segment.text
        
        # In ra màn hình dạng bảng
        print(start)
        f.write(start + "\n")
        print(end)
        f.write(end + "\n")
        print(text)
        f.write(text + "\n")
        
    
        # Lưu lại text để dùng cho việc khác (như Dịch thuật)
        full_text_log += text + " "

end_time = time.time()
print("-" * 50)
print(f"✅ Xử lý xong trong: {end_time - start_time:.2f} giây")
print("\n📝 TOÀN BỘ VĂN BẢN GỘP:")
print(full_text_log.strip())

🚀 Đang load mô hình large-v3 vào GPU T4...
⏳ Đang bắt đầu xử lý video (Transcribing)...

Detected language: zh (Độ tin cậy: 0.99)
--------------------------------------------------
TIME START | TIME END   | TEXT CONTENT
--------------------------------------------------
/kaggle/input/hoi-tuong-tuyet-vong/Tp 3 Hi tng tuyt vng Tp 3 Ln u gp g b ba ().mp4
0.00s
0.70s
你敢信
0.70s
2.16s
这个少年有多惨
2.16s
3.46s
穿越过来
3.46s
4.18s
没有系统
4.18s
5.18s
也没有戒指的奥秘
5.18s
6.77s
努力修到千年
6.77s
7.85s
苟到元英期
7.85s
9.37s
却陷入了一场危机
9.37s
11.39s
在灭世之前不断萎缩
11.39s
16.51s
一条不知修为的小蛇
16.51s
17.89s
速度之快
17.89s
19.09s
眨眼间
19.09s
20.95s
破掉了梦不从的护士身体
20.95s
21.57s
不
21.57s
22.71s
这不可能
22.71s
23.87s
七步之内
23.87s
25.49s
梦不从已难毙命
25.49s
27.11s
肉身毫无防备
27.11s
29.31s
剧毒迅速夺去了他的生命
29.31s
32.48s
荒谷森林危险至极
32.48s
34.00s
他堂堂元英期
34.00s
35.10s
大修士折在这地方
35.10s
35.96s
已有五六次
35.96s
37.60s
高空妖物太多
37.60s
39.26s
只能靠脚赶步
39.26s
40.94s
谁知经验不足
40.94s
42.54s
一处阴暗的沼泽
42.54s
43.96s
将他的双腿
43.96s
45.86s
腐蚀成赤裸裸的白骨
45.86s
47.00s
照这样下去
47.00s
48.44s
逃离巨爪危机的可能性
48.